# WiFi

Some microcontrollers come with built-in WiFi radios. Others use coprocessors for this purpose.

## Connecting

The built-in help for the ESP32 shows the code for connecting:

In [1]:
%connect esp32

help()

Connected to esp32 @ serial:///dev/ttyUSB0
Welcome to MicroPython on the ESP32!

For generic online docs please visit http://docs.micropython.org/

For access to the hardware use the 'machine' module:

import machine
pin12 = machine.Pin(12, machine.Pin.OUT)
pin12.value(1)
pin13 = machine.Pin(13, machine.Pin.IN, machine.Pin.PULL_UP)
print(pin13.value())
i2c = machine.I2C(scl=machine.Pin(21), sda=machine.Pin(22))
i2c.scan()
i2c.writeto(addr, b'1234')
i2c.readfrom(addr, 4)

Basic WiFi configuration:

import network
sta_if = network.WLAN(network.STA_IF); sta_if.active(True)
sta_if.scan()                             # Scan for available access points
sta_if.connect("<AP_name>", "<password>") # Connect to an AP
sta_if.isconnected()                      # Check for successful connection

Control commands:
  CTRL-A        -- on a blank line, enter raw REPL mode
  CTRL-B        -- on a blank line, enter normal REPL mode
  CTRL-C        -- interrupt a running program
  CTRL-D        -- on a blan

Modify the `connect` statement to use your `SSID` and `password` into the code. Rather than inserting them directly, I place them into a file, `$IOT_PROJECTS/libs/secrets.py` and upload that to the microcontroller.

Sample `secrets.py` with wifi credentials and other "secrets":

```python
# secrets.py

# wifi
wifi_ssid = 'MY_SSID'
wifi_pwd  = 'MY_PASSWORD'

# timezone
tz_offset = -8*3600    # PST

# webrepl password, 4 .. 9 characters
webrepl_pwd = 'er93xa'

# https://openweathermap.org/
openweathermap_apiid = "1c336ac61add531d32b73af8f2b"
```

In [1]:
%%writefile $IOT_PROJECTS/devices/example.yaml
esp32:
    uid: 30:ae:a4:30:84:34
    path: libs
    resources:
        - secrets.py

Writing /home/iot/iot49.org/docs/projects/devices/example.yaml


In [1]:
%rsync

Directories match


In [1]:
import network, secrets

wlan = network.WLAN(network.STA_IF); 
wlan.active(True)
wlan.connect(secrets.wifi_ssid, secrets.wifi_pwd)
print("connected:", wlan.isconnected())

connected: True


Let's check the IP address:

In [1]:
wlan = network.WLAN(network.STA_IF)
print(wlan.ifconfig()[0])

10.39.40.135


We can now ping the device:

In [1]:
!ping -c 3 10.39.40.135

PING 10.39.40.135 (10.39.40.135) 56(84) bytes of data.
64 bytes from 10.39.40.135: icmp_seq=1 ttl=255 time=1385 ms
64 bytes from 10.39.40.135: icmp_seq=2 ttl=255 time=328 ms
64 bytes from 10.39.40.135: icmp_seq=3 ttl=255 time=248 ms

--- 10.39.40.135 ping statistics ---
3 packets transmitted, 3 received, 0% packet loss, time 59ms
rtt min/avg/max/mdev = 247.953/653.696/1384.908/518.082 ms, pipe 2


## Internet time

Now that we are connected, we can fetch the current time from the internet. Let's check first what it is without initialization:

In [1]:
import time

print(time.gmtime())

(2021, 8, 1, 20, 27, 17, 6, 213)


Well, that was a while ago ...

Let's fix this.

In [1]:
import ntptime, time, machine, secrets

# fetch time from internet
tm = time.localtime(ntptime.time() + getattr(secrets, 'tz_offset', 0))
print("time", tm)

# set the time
machine.RTC().datetime((tm[0], tm[1], tm[2], tm[6] + 1, tm[3], tm[4], tm[5], 0))

time (2021, 8, 1, 20, 27, 19, 6, 213)


Formatted:

In [1]:
rtc = machine.RTC()
t = rtc.datetime()
print('{:04d}-{:02d}-{:02d} {:02d}:{:02d}'.format(t[0], t[1], t[2], t[4], t[5]))

2021-08-01 20:27


## Webrepl

Once the microcontroller is connected to the internet, we can program it wirelessly. Great for robot projects! Make sure that `webrepl_pwd` is defined in `secrets.py`, then create and upload `webrepl_cfg.py`.

In [ ]:
!mkdir -p $IOT_PROJECTS/internet/code

In [2]:
%%writefile $IOT_PROJECTS/internet/code/webrepl_cfg.py
from secrets import webrepl_pwd

PASS = webrepl_pwd

Writing /home/iot/iot49.org/docs/projects/internet/code/webrepl_cfg.py


In [1]:
%%writefile $IOT_PROJECTS/devices/example.yaml
esp32:
    uid: 30:ae:a4:30:84:34
    path: internet
    resources:
        - secrets.py:
            path: libs
        - code

Writing /home/iot/iot49.org/docs/projects/devices/example.yaml


In [1]:
%rsync

Directories match


Start the webrepl daemon on the microcontroller:

In [1]:
import webrepl
webrepl.start()

WebREPL daemon started on ws://10.39.40.135:8266
Started webrepl in normal mode


Advertise the webrepl with regular broadcast messages so *ide49* discovers it.

In [1]:
from socket import socket, AF_INET, SOCK_DGRAM
import _thread

def broadcaster(msg, port):
    so = socket(AF_INET, SOCK_DGRAM)
    try:
        # pre-allocate address (on heap)
        addr = ('255.255.255.255', port)
        while True:
            so.sendto(msg, addr)
            time.sleep(1)
    finally:
        so.close()

msg = "ws://{}:8266\n{}".format(
    network.WLAN(network.STA_IF).ifconfig()[0],
    ":".join("{:02x}".format(x) for x in machine.unique_id())
)
th = _thread.start_new_thread(broadcaster, (msg, getattr(secrets, 'broadcast_port', 50000)))

Connect wirelessly:

In [1]:
%discover

esp32  serial:///dev/ttyUSB0   
esp32  ws://10.39.40.135:8266  


In [1]:
%connect esp32 ws

print("connected to webrepl")

Connected to esp32 @ ws://10.39.40.135:8266
connected to webrepl


Unfortunately wireless connection via webrepl is slow. The primary reason is that webrepl lacks a feature to tell when it's ready to receive code and when not. *ide49* takes a conservative approach and sends code only in small chunks with pauses inbetween.

*Note:* if you rather not use the broadcaster code to advertise the webrepl, you can instead connect with the url:

In [1]:
%connect "ws://10.39.40.135:8266"

Connected to esp32 @ ws://10.39.40.135:8266


## boot.py

To automatically connect the ESP32 whenever it boots, place the instructions in a file `boot.py`. MicroPython executes this file whenever it starts.

In [1]:
%%writefile $IOT_PROJECTS/internet/code/boot.py
import micropython, machine, network, ntptime, time
import secrets

# handle errors in interrupts
micropython.alloc_emergency_exception_buf(100)

def connect():
    """connect to wifi, get ntp time, advertise hostname (if not None)"""
    wlan = network.WLAN(network.STA_IF)
    if wlan.isconnected(): return
    wlan.active(True)
    print("Connecting to WLAN ... ", end="")
    wlan.connect(getattr(secrets, 'wifi_ssid', 'SSID'),
                 getattr(secrets, 'wifi_pwd', ''))
    # wait for connection to be established ...
    for _ in range(30):
        if wlan.isconnected(): break
        time.sleep_ms(100)
    if not wlan.isconnected():
        print("Unable to connect to WiFi!")
        wlan.disconnect()
        return
    # set clock to local time
    tm = time.localtime(ntptime.time() + getattr(secrets, 'tz_offset', 0))
    print("time", tm)
    machine.RTC().datetime((tm[0], tm[1], tm[2], tm[6] + 1, tm[3], tm[4], tm[5], 0))
    return 

# connect to WiFi
connect()

if hasattr(secrets, 'webrepl_pwd'):
    # start webrepl
    import webrepl
    webrepl.start()

    # advertise
    from socket import socket, AF_INET, SOCK_DGRAM
    import _thread

    def broadcaster(msg, port):
        so = socket(AF_INET, SOCK_DGRAM)
        try:
            # pre-allocate address (on heap)
            addr = ('255.255.255.255', port)
            while True:
                so.sendto(msg, addr)
                time.sleep(1)
        finally:
            so.close()

    msg = "ws://{}:8266\n{}".format(
        network.WLAN(network.STA_IF).ifconfig()[0],
        ":".join("{:02x}".format(x) for x in machine.unique_id())
    )
    th = _thread.start_new_thread(broadcaster, (msg, getattr(secrets, 'broadcast_port', 50000)))

Writing /home/iot/iot49.org/docs/projects/internet/code/boot.py


In [1]:
%rsync
%softreset

UPDATE  /boot.py

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!   softreset ...     !!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!



In [1]:
# switch back to serial connection
%connect esp32 serial

Connected to esp32 @ serial:///dev/ttyUSB0


In [1]:
# and again to wireless
%connect esp32 ws

Connected to esp32 @ ws://10.39.40.135:8266


Try operating the microcontroller from a battery or power-only USB cable to verify that connection is indeed wireless. 